## Set up

In [1]:
import pickle
from enum import Enum
from dataclasses import dataclass
from pathlib import Path

import keras as tfk
import numpy as np
import tensorflow as tf
from keras import layers as tfkl
from tensorflow.data import Dataset as tfds


2024-11-15 12:57:35.025153: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-15 12:57:35.025406: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-15 12:57:35.028221: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-15 12:57:35.062026: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-15 12:57:35.729134: W tensorflow/compiler/tf2tensorrt/utils/py_uti

In [2]:
print(f"Keras version is {tfk.__version__}")

Keras version is 3.3.3


In [3]:
AUGMENTED_DIR = Path().absolute().parent / "data" / "augmented"
OUTPUT_DIR = Path().absolute().parent / "submission"
# If the data is not there, then we're running in Kaggle
if not AUGMENTED_DIR.exists():
    AUGMENTED_DIR = Path("/kaggle/input/an2dl-homework-1-augmented/augmented")
    OUTPUT_DIR = Path()

TRAIN_TFDS_DIR = AUGMENTED_DIR / "train"
VAL_TFDS_DIR = AUGMENTED_DIR / "val"

print(TRAIN_TFDS_DIR)
print(VAL_TFDS_DIR)
print(OUTPUT_DIR)

/home/tomaz/git/Politecnico/Subjects/deep-learning/an2dl/homework-1/data/augmented/train
/home/tomaz/git/Politecnico/Subjects/deep-learning/an2dl/homework-1/data/augmented/val
/home/tomaz/git/Politecnico/Subjects/deep-learning/an2dl/homework-1/submission


In [4]:
SEED = 42
KERAS_VERBOSITY = 2
BATCH_SIZE = 128
LOSS = "categorical_crossentropy"
METRICS = ["accuracy", "precision", "recall", "f1_score"]

In [5]:
tfk.utils.set_random_seed(SEED)

In [6]:
@dataclass
class Hyperparameters:
    # Model
    global_pooling_type = "avg"
    regularisations = (
        (0, 1e-3),
        (0, 1e-2),
    )
    layer_units = (256, 64)
    dropout = 0.6
    activation = "silu"
    class_weight = {
        0: 1.0,
        1: 1.0,
        2: 1.0,
        3: 2.0,
        4: 1.25,
        5: 1.5,
        6: 1.25,
        7: 1.0,
    }
    # Training
    optimiser = tfk.optimizers.AdamW
    learning_rate = 1e-4
    epochs = 500
    ## Early stopping parameters
    es_patience = 10
    es_min_delta = 1e-3
    ## Learning rate schedule
    lr_patience = 5
    lr_min_delta = 1e-2
    lr_decay_factor = 0.1
    min_lr = 1e-7

hp = Hyperparameters()

In [7]:
class EmbeddingWidth(Enum):
    EfficientNetV2B3 = 1536

### Load

In [8]:
train_dataset = tfds.load(str(TRAIN_TFDS_DIR)).batch(BATCH_SIZE)
val_dataset = tfds.load(str(VAL_TFDS_DIR)).batch(BATCH_SIZE)

train_dataset

<_BatchDataset element_spec=(TensorSpec(shape=(None, 96, 96, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(None, 8), dtype=tf.uint8, name=None))>

## Model

In [9]:
def build(hp: Hyperparameters) -> tfk.Model:
    inputs = tfkl.Input((96, 96, 3))
    x = inputs

    # ImageNet images are 224x224 so we need to resize to use the
    # pre-trained backbone feature extractor
    x = tfkl.Resizing(224, 224)(x)

    # TODO Investigate changing `input_shape` to actual image shape
    # instead of resizing
    feature_extractor = tfk.applications.MobileNetV3Large(
        input_shape=(224, 224, 3),
        include_top=False,
        weights="imagenet",
        pooling=hp.global_pooling_type,
    )
    feature_extractor.trainable = False
    x = feature_extractor(x)

    for units, regularisation in zip(hp.layer_units, hp.regularisations):
        x = tfkl.Dense(
            units,
            activation=hp.activation,
            kernel_regularizer=tfk.regularizers.L1L2(*regularisation)
        )(x)

    x = tfkl.Dropout(hp.dropout)(x)

    # Classification problem has 8 output classes
    # so the final layer has 8 neurons
    # with a softmax activation
    outputs = tfkl.Dense(8, activation="softmax")(x)

    return tfk.Model(inputs, outputs)

In [10]:
model = build(hp)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 96, 96, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resizing (Resizing)             │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ MobilenetV3large (Functional)   │ (None, 960)            │     2,996,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       246,016 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 8)              │           520 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,259,336 (12.43 MB)

 Trainable params: 262,984 (1.00 MB)

 Non-trainable params: 2,996,352 (11.43 MB)

In [11]:
def fit(hp: Hyperparameters, train: tf.data.Dataset, val: tf.data.Dataset) -> tuple[tfk.Model, tfk.callbacks.History]:
    model = build(hp)
    model.compile(
        optimizer=hp.optimiser(hp.learning_rate),
        loss=LOSS,
        metrics=METRICS,
    )
    history = model.fit(
        train.prefetch(tf.data.AUTOTUNE),
        epochs=hp.epochs,
        validation_data=val.prefetch(tf.data.AUTOTUNE),
        class_weight=hp.class_weight,
        callbacks=[
            tfk.callbacks.EarlyStopping(
                min_delta=hp.es_min_delta,
                patience=hp.es_patience,
                restore_best_weights=True,
                verbose=1,
            ),
            tfk.callbacks.ReduceLROnPlateau(
                factor=hp.lr_decay_factor,
                patience=hp.lr_patience,
                min_delta=hp.lr_min_delta,
                min_lr=hp.min_lr,
                verbose=1,
            )
        ],
        verbose=KERAS_VERBOSITY,
    )
    return model, history

In [12]:
model, history = fit(hp, train_dataset, val_dataset)

Epoch 1/500


KeyboardInterrupt: 

## Save results

In [ ]:
model.save(OUTPUT_DIR / "model.keras")

In [ ]:
with (OUTPUT_DIR / "history.pkl").open("wb") as f:
    pickle.dump(history, f)

In [ ]:
np.save(
    OUTPUT_DIR / "train_predictions",
    model.predict(train_dataset, verbose=KERAS_VERBOSITY),
)
np.save(
    OUTPUT_DIR / "val_predictions",
    model.predict(val_dataset, verbose=KERAS_VERBOSITY),
)